# BUNDAS ANALYSIS

# Group Members
- Jeovine Oruko
- Dennis Ogunde
- Thiga Mureithi
- Brenda Cherono

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeClassifier

In [2]:
bundas_train = pd.read_csv("bundas_train.csv")
bundas_train.head()

,Item_ID,Weight,FatContent,Visibility,Category,Max_Price,Store_ID,Store_Establishment_Year,Store_Size,Store_Location_Type,Store_Type,Item_Store_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [3]:
bundas_train.describe(include='object')

,Item_ID,FatContent,Category,Store_ID,Store_Size,Store_Location_Type,Store_Type
count,8523,8523,8523,8523,6113,8523,8523
unique,1559,5,16,10,3,3,4
top,FDW13,Low Fat,Fruits and Vegetables,OUT027,Medium,Tier 3,Supermarket Type1
freq,10,5089,1232,935,2793,3350,5577


In [4]:
bundas_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
Item_ID                     8523 non-null object
Weight                      7060 non-null float64
FatContent                  8523 non-null object
Visibility                  8523 non-null float64
Category                    8523 non-null object
Max_Price                   8523 non-null float64
Store_ID                    8523 non-null object
Store_Establishment_Year    8523 non-null int64
Store_Size                  6113 non-null object
Store_Location_Type         8523 non-null object
Store_Type                  8523 non-null object
Item_Store_Sales            8523 non-null float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.1+ KB


In [5]:
bundas_train.shape

(8523, 12)

## Data Cleaning

In [6]:
bundas_train.isnull().any()

Item_ID                     False
Weight                       True
FatContent                  False
Visibility                  False
Category                    False
Max_Price                   False
Store_ID                    False
Store_Establishment_Year    False
Store_Size                   True
Store_Location_Type         False
Store_Type                  False
Item_Store_Sales            False
dtype: bool

### Weight and Store Size columns have missing data

In [7]:
bundas_train.isnull().sum()

Item_ID                        0
Weight                      1463
FatContent                     0
Visibility                     0
Category                       0
Max_Price                      0
Store_ID                       0
Store_Establishment_Year       0
Store_Size                  2410
Store_Location_Type            0
Store_Type                     0
Item_Store_Sales               0
dtype: int64

In [8]:
bundas_train[['Store_Size']].mode()

,Store_Size
0,Medium


### Fill null values in Store Size column with 'Unknown'

In [9]:
bundas_train['Store_Size'] = bundas_train.Store_Size.fillna('Unknown')

### Null values in Store Size columns have been dealt with by assigning 'Unknown' to them.

In [10]:
bundas_train[['Store_Size']].isnull().sum()

Store_Size    0
dtype: int64

In [11]:
weight = bundas_train[['Weight']]
float(weight.mean())

12.857645184136183

### Fill null values in Weight column with the mean which is 12.857645

In [12]:
bundas_train[['Weight']] = bundas_train.Weight.fillna(float(weight.mean()))

In [13]:
bundas_train[['Weight']].isnull().sum()

Weight    0
dtype: int64

In [14]:
bundas_train.sample(10)

,Item_ID,Weight,FatContent,Visibility,Category,Max_Price,Store_ID,Store_Establishment_Year,Store_Size,Store_Location_Type,Store_Type,Item_Store_Sales
4167,FDT40,5.985,Low Fat,0.095990,Frozen Foods,127.3678,OUT045,2002,Unknown,Tier 2,Supermarket Type1,508.6712
6681,DRK11,8.210,Low Fat,0.010781,Hard Drinks,149.9392,OUT049,1999,Medium,Tier 1,Supermarket Type1,1938.8096
5376,FDT32,19.000,Regular,0.065622,Fruits and Vegetables,186.5214,OUT035,2004,Small,Tier 2,Supermarket Type1,5275.7992
2689,NCX06,17.600,Low Fat,0.015687,Household,182.6976,OUT046,1997,Small,Tier 1,Supermarket Type1,3440.8544
6465,FDY15,18.250,Regular,0.171794,Dairy,154.6630,OUT017,2007,Unknown,Tier 2,Supermarket Type1,2659.8710
2333,FDI41,18.500,Regular,0.062257,Frozen Foods,148.0418,OUT046,1997,Small,Tier 1,Supermarket Type1,2059.9852
1091,FDV22,14.850,Regular,0.009931,Snack Foods,157.4630,OUT013,1987,High,Tier 3,Supermarket Type1,2503.4080
6137,FDZ43,11.000,Regular,0.057059,Fruits and Vegetables,241.7512,OUT046,1997,Small,Tier 1,Supermarket Type1,3635.2680
6570,FDP24,20.600,Low Fat,0.083172,Baking Goods,121.0756,OUT045,2002,Unknown,Tier 2,Supermarket Type1,1696.4584
4452,FDI24,10.300,Low Fat,0.078678,Baking Goods,177.0370,OUT013,1987,High,Tier 3,Supermarket Type1,3705.1770


In [15]:
bundas_train.FatContent.unique()

array(['Low Fat', 'Regular', 'low fat', 'LF', 'reg'], dtype=object)

### Fat Content column has multiple values that are semantically the same thing, e.g Regular = reg

In [16]:
bundas_train.FatContent.replace('low fat', 'Low Fat', inplace=True)
bundas_train.FatContent.replace('LF', 'Low Fat', inplace=True)
bundas_train.FatContent.replace('reg', 'Regular', inplace=True)

In [17]:
bundas_train.Category.unique()

array(['Dairy', 'Soft Drinks', 'Meat', 'Fruits and Vegetables',
       'Household', 'Baking Goods', 'Snack Foods', 'Frozen Foods',
       'Breakfast', 'Health and Hygiene', 'Hard Drinks', 'Canned',
       'Breads', 'Starchy Foods', 'Others', 'Seafood'], dtype=object)

In [18]:
bundas_train.Store_Type.unique()

array(['Supermarket Type1', 'Supermarket Type2', 'Grocery Store',
       'Supermarket Type3'], dtype=object)

In [19]:
bundas_train.Store_Location_Type.unique()

array(['Tier 1', 'Tier 3', 'Tier 2'], dtype=object)

In [20]:
bundas_train[['Category', 'Store_Type']].sample(20)

,Category,Store_Type
5837,Frozen Foods,Grocery Store
6126,Health and Hygiene,Supermarket Type2
8335,Breakfast,Supermarket Type1
7172,Household,Supermarket Type1
6420,Fruits and Vegetables,Supermarket Type1
5542,Meat,Grocery Store
5061,Baking Goods,Supermarket Type1
8505,Snack Foods,Supermarket Type2
852,Canned,Supermarket Type1
7375,Meat,Supermarket Type2


In [21]:
bundas_train.Visibility.max()

0.328390948

## Feature Engineering

### Some columns are meaningless in this analysis so they will be dropped. We will also separate the features from the target variable

In [22]:
X_train = bundas_train.drop(['Item_ID', 'Item_Store_Sales'],axis=1)
y_train = bundas_train['Item_Store_Sales']
X_train

,Weight,FatContent,Visibility,Category,Max_Price,Store_ID,Store_Establishment_Year,Store_Size,Store_Location_Type,Store_Type
0,9.300000,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,5.920000,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2
2,17.500000,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1
3,19.200000,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Unknown,Tier 3,Grocery Store
4,8.930000,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1
5,10.395000,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2
6,13.650000,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1
7,12.857645,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3
8,16.200000,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,Unknown,Tier 2,Supermarket Type1
9,19.200000,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,Unknown,Tier 2,Supermarket Type1


### Now we need to encode the categorical variables

In [23]:
label_encoder = LabelEncoder()
# one_hot_encoder = OneHotEncoder(sparse=False)
# encoded_fat_content = one_hot_encoder.fit_transform(X_train[["FatContent"]])
# encoded_category = one_hot_encoder.fit_transform(X_train[["Category"]])
# encoded_store_size = one_hot_encoder.fit_transform(X_train[["Store_Size"]])
# encoded_store_location_type = one_hot_encoder.fit_transform(X_train[["Store_Location_Type"]])
# encoded_store_type = one_hot_encoder.fit_transform(X_train[["Store_Type"]])
# encoded_fat_content_df = pd.DataFrame(encoded_fat_content)

# X_train_features = pd.DataFrame(encoded_fat_content
X_train_features = pd.get_dummies(X_train)
# X_train_features.drop(['Weight', 'Visibility', 'Max_Price', 'Store_Establishment_Year'], axis = 1, inplace=True)
X_train_features.dtypes

Weight                            float64
Visibility                        float64
Max_Price                         float64
Store_Establishment_Year            int64
FatContent_Low Fat                  uint8
FatContent_Regular                  uint8
Category_Baking Goods               uint8
Category_Breads                     uint8
Category_Breakfast                  uint8
Category_Canned                     uint8
Category_Dairy                      uint8
Category_Frozen Foods               uint8
Category_Fruits and Vegetables      uint8
Category_Hard Drinks                uint8
Category_Health and Hygiene         uint8
Category_Household                  uint8
Category_Meat                       uint8
Category_Others                     uint8
Category_Seafood                    uint8
Category_Snack Foods                uint8
Category_Soft Drinks                uint8
Category_Starchy Foods              uint8
Store_ID_OUT010                     uint8
Store_ID_OUT013                   

## Model

In [24]:
model = linear_model.LinearRegression()
model.fit(X = X_train_features, y = y_train)
y_pred = model.predict(X_train_features)
r2_score(y_train, y_pred)

0.5637650620863444

In [25]:
bundas_test = pd.read_csv('bundas_test.csv')
print(bundas_test)
bundas_test.isnull().any()
bundas_test['Store_Size'] = bundas_test.Store_Size.fillna('Unknown')
bundas_test[['Weight']] = bundas_test.Weight.fillna(float(weight.mean()))
bundas_test.isnull().any()

     Item_ID  Weight FatContent  Visibility               Category  Max_Price  \
0      FDW58  20.750    Low Fat    0.007565            Snack Foods   107.8622   
1      FDW14   8.300        reg    0.038428                  Dairy    87.3198   
2      NCN55  14.600    Low Fat    0.099575                 Others   241.7538   
3      FDQ58   7.315    Low Fat    0.015388            Snack Foods   155.0340   
4      FDY38     NaN    Regular    0.118599                  Dairy   234.2300   
5      FDH56   9.800    Regular    0.063817  Fruits and Vegetables   117.1492   
6      FDL48  19.350    Regular    0.082602           Baking Goods    50.1034   
7      FDC48     NaN    Low Fat    0.015782           Baking Goods    81.0592   
8      FDN33   6.305    Regular    0.123365            Snack Foods    95.7436   
9      FDA36   5.985    Low Fat    0.005698           Baking Goods   186.8924   
10     FDT44  16.600    Low Fat    0.103569  Fruits and Vegetables   118.3466   
11     FDQ56   6.590    Low 

Item_ID                     False
Weight                      False
FatContent                  False
Visibility                  False
Category                    False
Max_Price                   False
Store_ID                    False
Store_Establishment_Year    False
Store_Size                  False
Store_Location_Type         False
Store_Type                  False
dtype: bool

In [26]:
bundas_test['FatContent'].unique()

array(['Low Fat', 'reg', 'Regular', 'LF', 'low fat'], dtype=object)

In [27]:
bundas_test.FatContent.replace('low fat', 'Low Fat', inplace=True)
bundas_test.FatContent.replace('LF', 'Low Fat', inplace=True)
bundas_test.FatContent.replace('reg', 'Regular', inplace=True)
bundas_test.head()

,Item_ID,Weight,FatContent,Visibility,Category,Max_Price,Store_ID,Store_Establishment_Year,Store_Size,Store_Location_Type,Store_Type
0,FDW58,20.750000,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300000,Regular,0.038428,Dairy,87.3198,OUT017,2007,Unknown,Tier 2,Supermarket Type1
2,NCN55,14.600000,Low Fat,0.099575,Others,241.7538,OUT010,1998,Unknown,Tier 3,Grocery Store
3,FDQ58,7.315000,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,Unknown,Tier 2,Supermarket Type1
4,FDY38,12.857645,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [31]:
X_test = bundas_test.drop(['Item_ID'],axis=1)
X_test_features = pd.get_dummies(X_test)
model.predict(X_test_features)

array([1835.76091502, 1531.50296273, 1881.66025009, ..., 1824.37530058,
       3587.37688271, 1285.68376058])